In [1]:
import os
import sys
import json
import yaml

In [15]:
from frama_c import FramaCBenchmark, InvalidBenchmarkException

In [10]:
benchmarks = json.load(open("svcomp_features.json", "r", encoding="utf-8"))

In [24]:
ultimate_failure_file = open("ultimate_failed_benchmarks.txt", "r", encoding="utf-8").read().splitlines()

In [25]:
ultimate_failure_file

['../../sv-benchmarks/c/array-examples/data_structures_set_multi_proc_ground-2.c',
 '../../sv-benchmarks/c/array-examples/relax-2-2.c',
 '../../sv-benchmarks/c/array-examples/sanfoundry_02_ground.c',
 '../../sv-benchmarks/c/array-examples/sanfoundry_10_ground.c',
 '../../sv-benchmarks/c/array-examples/sorting_selectionsort_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copy2_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copy3_ground-1.c',
 '../../sv-benchmarks/c/array-examples/standard_copy4_ground-1.c',
 '../../sv-benchmarks/c/array-examples/standard_copy5_ground-1.c',
 '../../sv-benchmarks/c/array-examples/standard_copy6_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copy7_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copy8_ground-1.c',
 '../../sv-benchmarks/c/array-examples/standard_copy9_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copyInitSum2_ground-2.c',
 '../../sv-benchmarks/c/array-examples/standard_copy

In [29]:
target_files = []
bigger_set_target_files = []

for benchmark in benchmarks:
    if benchmark["num_loops"] is None:
        continue
    if (benchmark["num_loops"] > 1 or benchmark["more_than_one_method"]) and not benchmark["uses_arrays"] and not benchmark["uses_pointers"] and benchmark["num_lines"] < 500:
        bigger_set_target_files.append(benchmark["file"])
        if benchmark["file"] in ultimate_failure_file:
            target_files.append(benchmark["file"])

In [30]:
len(target_files), len(bigger_set_target_files)

(233, 564)

In [11]:
s00, s01, s10, s11, s20, s21 = 0, 0, 0, 0, 0, 0

for benchmark in benchmarks:
    if "num_loops" not in benchmark or benchmark["num_loops"] is None or benchmark["uses_arrays"] or benchmark["uses_pointers"]:
        # print(benchmark)
        continue
    if benchmark["num_loops"] == 0 and not benchmark["more_than_one_method"]:
        s00 += 1
    elif benchmark["num_loops"] == 0 and benchmark["more_than_one_method"]:
        s01 += 1
    elif benchmark["num_loops"] == 1 and not benchmark["more_than_one_method"]:
        s10 += 1
    elif benchmark["num_loops"] == 1 and benchmark["more_than_one_method"]:
        s11 += 1
    elif benchmark["num_loops"] > 1 and not benchmark["more_than_one_method"]:
        s20 += 1
    elif benchmark["num_loops"] > 1 and benchmark["more_than_one_method"]:
        s21 += 1

In [12]:
s00, s01, s10, s11, s20, s21

(192, 143, 1658, 1407, 201, 693)

In [16]:
s00 + s01 + s10 + s11 + s20 + s21

4294

In [8]:
no_loop = {}
one_loop = {}
multiple_loops = {}

for values in benchmarks:
    if values["num_loops"] == 0:
        no_loop[values["file"]] = values
    elif values["num_loops"] == 1:
        one_loop[values["file"]] = values
    else:
        multiple_loops[values["file"]] = values

In [9]:
valid_1, invalid_1 = 0, 0
valid_2, invalid_2 = 0, 0
valid_3, invalid_3 = 0, 0

for key, values in no_loop.items():
    if not values["array_or_pointers"]:
        if values["num_methods"] == 1:
            valid_1 += 1
        elif values["num_methods"] > 1:
            invalid_1 += 1
    else:
        continue

for key, values in one_loop.items():
    if not values["array_or_pointers"]:
        if values["num_methods"] == 1:
            valid_2 += 1
        elif values["num_methods"] > 1:
            invalid_2 += 1
    else:
        continue

for key, values in multiple_loops.items():
    if not values["array_or_pointers"]:
        if values["num_methods"] == 1:
            valid_3 += 1
        elif values["num_methods"] > 1:
            invalid_3 += 1
    else:
        continue


KeyError: 'array_or_pointers'

In [43]:
valid_1, invalid_1, valid_2, invalid_2, valid_3, invalid_3

(36, 138, 535, 31, 205, 16)

In [44]:
valid_1 + invalid_1 + valid_2 + invalid_2 + valid_3 + invalid_3

961

In [ ]:
benchmark_features = {}
for benchmark in benchmarks:
    try:
        frama_c_benchmark = FramaCBenchmark()
        benchmark_features[benchmark] = frama_c_benchmark.get_features()
    except InvalidBenchmarkException:
        print("Invalid benchmark: " + benchmark)

In [3]:
svcomp_c_dir = os.path.join(os.getcwd(), "../../sv-benchmarks/c/")

In [4]:
# walk through the svcomp directory and find all the c files
svcomp_c_files = []
svcomp_yml_files = []
for root, dirs, files in os.walk(svcomp_c_dir):
    for file in files:
        if file.endswith(".c"):
            svcomp_c_files.append(os.path.join(root, file))
        elif file.endswith(".yml"):
            svcomp_yml_files.append(os.path.join(root, file))


In [5]:
temp_file = open(svcomp_yml_files[1]).read().split("\n")

In [6]:
temp_file = open(svcomp_c_files[1]).read().split("\n")

In [7]:
benchmark_yamls = []
for yml_file in svcomp_yml_files:
    benchmark_yamls.append((yml_file, yaml.load(open(yml_file, "r"), Loader=yaml.FullLoader)))

In [8]:
benchmarks_with_reachability = []
for yml_file, benchmark_yaml in benchmark_yamls:
    if any([x["property_file"] == "../properties/unreach-call.prp" for x in benchmark_yaml["properties"]]):
        if any([x["property_file"] == "../properties/unreach-call.prp" and x["expected_verdict"] for x in benchmark_yaml["properties"]]):
            benchmarks_with_reachability.append((yml_file, benchmark_yaml))

In [39]:
len(benchmarks_with_reachability)

10887

In [10]:
benchmarks_with_only_reachability = []
for yml_file, benchmark_yaml in benchmarks_with_reachability:
    if len(benchmark_yaml["properties"]) == 1:
        benchmarks_with_only_reachability.append((yml_file, benchmark_yaml))

In [11]:
len(benchmarks_with_only_reachability)

6804

In [43]:
json.dump(new_bench, open("filtered_benchmarks.json", "w"), indent=4)

In [40]:
new_bench = []
for b in benchmarks_with_reachability:
    if "-todo\\" in b[0]:
        continue
    if b[1]["input_files"][-2:] == ".i":
        if not os.path.exists(b[0][:-4] + ".c"):
            print(b[1]["input_files"])
            continue
    new_bench.append(b)

aws_add_size_checked_harness.i
aws_add_size_saturating_harness.i
aws_array_eq_c_str_harness.i
aws_array_eq_c_str_ignore_case_harness.i
aws_array_eq_harness.i
aws_array_eq_ignore_case_harness.i
aws_array_list_back_harness.i
aws_array_list_capacity_harness.i
aws_array_list_clean_up_harness.i
aws_array_list_clear_harness.i
aws_array_list_comparator_string_harness.i
aws_array_list_copy_harness.i
aws_array_list_ensure_capacity_harness.i
aws_array_list_erase_harness.i
aws_array_list_front_harness.i
aws_array_list_get_at_harness.i
aws_array_list_get_at_ptr_harness.i
aws_array_list_init_dynamic_harness.i
aws_array_list_init_static_harness.i
aws_array_list_length_harness.i
aws_array_list_pop_back_harness.i
aws_array_list_pop_front_harness.i
aws_array_list_pop_front_n_harness.i
aws_array_list_push_back_harness.i
aws_array_list_set_at_harness.i
aws_array_list_shrink_to_fit_harness.i
aws_array_list_sort_harness.i
aws_array_list_swap_contents_harness.i
aws_array_list_swap_harness.i
aws_byte_buf_adv

In [44]:
len(new_bench)

10583

In [82]:
import pandas as pd
import json

In [84]:
ground_truth_files = json.load(open("../../svcomp_reachability_features.json", "r"))

In [85]:
ground_truth = {}
for benchmark in ground_truth_files["rows"]:
    file_name = "../" + benchmark["href"][:-4] + ".c"
    ground_truth[file_name] = benchmark["id"][1]

In [13]:
svcomp_files = json.load(open("svcomp_features.json", "r", encoding="utf-8"))

In [15]:
ultimate_failure_file = open("ultimate_failed_benchmarks.txt", "r").read().split("\n")
all_failure_file = open("all_tools_failed.txt", "r").read().split("\n")

In [93]:
num_solvers_json = json.load(open("number_of_tools_solving_svcomp.json", "r"))

In [94]:
final_json = []

for file in svcomp_files:
    new_file = file
    ground_truth_val = None
    num_solvers = None
    if file["file"] in ground_truth:
        ground_truth_val = ground_truth[file["file"]] == "true"
        num_solvers = num_solvers_json[file["file"]]
    else:
        i_file = file["file"][:-2] + ".c"
        ground_truth_val = ground_truth[i_file] == "true"
        num_solvers = num_solvers_json[i_file]
    new_file["Ground Truth"] = ground_truth_val
    new_file["Number of Solvers"] = num_solvers
    new_file["Ultimate Automizer Fails"] = (file["file"] in ultimate_failure_file)
    new_file["All Tools Fail"] = (file["file"] in all_failure_file)
    final_json.append(new_file)

In [97]:
final_json[3]

{'file': '../../sv-benchmarks/c/array-examples/sanfoundry_02_ground.c',
 'num_loops': 4,
 'more_than_one_method': False,
 'uses_arrays': True,
 'uses_pointers': False,
 'num_lines': 58,
 'Ground Truth': True,
 'Number of Solvers': 2,
 'Ultimate Automizer Fails': True,
 'All Tools Fail': False}

In [98]:
with open("all_svcomp_features.json", "w") as f:
    json.dump(final_json, f, indent=4)

In [22]:
# skip first index row
svcomp_excel = pd.DataFrame(final_json, index=None)


svcomp_excel.to_excel("svcomp_features_latest.xlsx")